In [1]:
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import os

from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.
    """
    # Convert to Path object if it's a string
    path_obj: Path = Path(path) if isinstance(path, str) else path

    # Get the parent directory and create it if it doesn't exist
    path_obj.parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

from settings import refresh_settings  # noqa: E402

settings = refresh_settings()

/Users/neidu/Desktop/Projects/Personal/RAG-Tutorials


## Extract

## [Simple Loader](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents/#defining-documents)

In [4]:
from llama_index.core import SimpleDirectoryReader

fp: str = "../data"
docs = SimpleDirectoryReader(input_dir=fp).load_data()

# console.print(docs)
docs

[Document(id_='b537fd39-08f3-414b-b670-c50f81b8de90', embedding=None, metadata={'page_label': '1', 'file_name': 'chelsea_transfer_news.pdf', 'file_path': '/Users/neidu/Desktop/Projects/Personal/RAG-Tutorials/notebooks/../data/chelsea_transfer_news.pdf', 'file_type': 'application/pdf', 'file_size': 604575, 'creation_date': '2025-07-25', 'last_modified_date': '2025-07-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Friday 25 July 2025 14:39, UK\nChelsea transfer news, rumours and\ngossip: Live updates and latest on\ndeals, signings, loans and contracts\nLatest Chelsea news\xa0\nSort by: Latest Oldest\nIn full: Chelsea 2025/26 Premier L

In [5]:
len(docs)

22

## Transform

### [Modify The Metadata](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents/#summary)

- Adjust the metadata before it's sent to the embedding model

In [6]:
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

document = Document(
    text="This is a sample document.",
    metadata={
        "file_name": "example.txt",
        "author": "Neidu E.",
        "number": 42,
        "date": "2025-07-25",
        "category": "example",
    },
    excluded_embed_metadata_keys=["file_name"],
    excluded_llm_metadata_keys=["file_name", "number"],
    metadata_separator="\n",
    metadata_template="{key}:{value}",
    text_template="Metadata: \n{metadata_str}\n-----\n\nContent: {content}",
)

# Embedding model sees this:
console.print(document.get_content(metadata_mode=MetadataMode.EMBED))

# LLM model sees this:
console.print(document.get_content(metadata_mode=MetadataMode.LLM))

Metadata: 
author:Neidu E.
number:42
date:2025-07-25
category:example
-----

Content: This is a sample document.

Metadata: 
author:Neidu E.
date:2025-07-25
category:example
-----

Content: This is a sample document.

In [7]:
console.print(docs[0].get_content(metadata_mode=MetadataMode.EMBED))

page_label: 1
file_path: /Users/neidu/Desktop/Projects/Personal/RAG-Tutorials/notebooks/../data/chelsea_transfer_news.pdf

Friday 25 July 2025 14:39, UK
Chelsea transfer news, rumours and
gossip: Live updates and latest on
deals, signings, loans and contracts
Latest Chelsea news 
Sort by: Latest Oldest
In full: Chelsea 2025/26 Premier League fixtures 
Transfer Centre LIVE! Deals, rumours, news on your phone
Download the Sky Sports app for Chelsea transfers, analysis and FREE highlights from
EVERY Premier League game   View post
24 Jul
16:22 Simons to Chelsea? The key questions answered...
Chelsea have held talks over signing RB Leipzig forward Xavi Simons - but how
much will they have to pay and who else want him? 
Sky Germany’s Leipzig reporter Philipp Hinze answers the key questions
around the deal. 
Keep scrolling! 
UPDATE
F o o t b al l 
 News Watch Scores & FixturesTables Transfers More
25/07/2025, 19:28 Chelsea transfer news, rumours and gossip: Live updates and latest on deals, signings, loans and 
contracts | Football News | Sky Sports
https://www.skysports.com/football/live-blog/11668/13025497/chelsea-transfer-news-rumours-and-gossip-live-updates-a
nd-latest-on-deals-signings-loans-and-… 1/17

In [8]:
console.print(docs[0])

Document(
    id_='b537fd39-08f3-414b-b670-c50f81b8de90',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'chelsea_transfer_news.pdf',
        'file_path': 
'/Users/neidu/Desktop/Projects/Personal/RAG-Tutorials/notebooks/../data/chelsea_transfer_news.pdf',
        'file_type': 'application/pdf',
        'file_size': 604575,
        'creation_date': '2025-07-25',
        'last_modified_date': '2025-07-25'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    metadata_template='{key}: {value}',
    metadata_separator='\n',
    text_resource=MediaResource(
        embeddings=None,
        data=None,
        text='Friday 25 July 2025 14:39, UK\nChelsea transfer news, rumours and\ngossip: Live updates and latest 
on\ndeals, signings, loans and contracts\nLatest Chelsea news\xa0\nSort by: Latest Oldest\nIn full: Chelsea 2025/26
Premier League fixtures\xa0\nTransfer Centre LIVE! Deals, rumours, news on your phone\nDownload the Sky Sports app 
for Chelsea transfers, analysis and FREE highlights from\nEVERY Premier League game\xa0\xa0\xa0View post\n24 
Jul\n16:22 Simons to Chelsea? The key questions answered...\nChelsea have held talks over signing RB Leipzig 
forward Xavi Simons - but how\nmuch will they have to pay and who else want him?\xa0\nSky Germany’s Leipzig 
reporter Philipp Hinze answers the key questions\naround the deal.\xa0\nKeep scrolling!\xa0\nUPDATE\nF o o t b al l
\n News Watch Scores & FixturesTables Transfers More\n25/07/2025, 19:28 Chelsea transfer news, rumours and gossip: 
Live updates and latest on deals, signings, loans and contracts | Football News | Sky 
Sports\nhttps://www.skysports.com/football/live-blog/11668/13025497/chelsea-transfer-news-rumours-and-gossip-live-u
pdates-and-latest-on-deals-signings-loans-and-… 1/17',
        path=None,
        url=None,
        mimetype=None
    ),
    image_resource=None,
    audio_resource=None,
    video_resource=None,
    text_template='{metadata_str}\n\n{content}'
)

### Update The Docs' Metadata

In [9]:
for doc in docs:
    # Define the metadata template
    doc.text_template = "Metadata: \n{metadata_str}\n-----\n\nContent: {content}"

    # Exclude the page_label from embedding
    if "page_label" not in doc.excluded_embed_metadata_keys:
        doc.excluded_embed_metadata_keys.append("page_label")

In [10]:
# Verify the changes
console.print(docs[0].get_content(metadata_mode=MetadataMode.EMBED))

Metadata: 
file_path: /Users/neidu/Desktop/Projects/Personal/RAG-Tutorials/notebooks/../data/chelsea_transfer_news.pdf
-----

Content: Friday 25 July 2025 14:39, UK
Chelsea transfer news, rumours and
gossip: Live updates and latest on
deals, signings, loans and contracts
Latest Chelsea news 
Sort by: Latest Oldest
In full: Chelsea 2025/26 Premier League fixtures 
Transfer Centre LIVE! Deals, rumours, news on your phone
Download the Sky Sports app for Chelsea transfers, analysis and FREE highlights from
EVERY Premier League game   View post
24 Jul
16:22 Simons to Chelsea? The key questions answered...
Chelsea have held talks over signing RB Leipzig forward Xavi Simons - but how
much will they have to pay and who else want him? 
Sky Germany’s Leipzig reporter Philipp Hinze answers the key questions
around the deal. 
Keep scrolling! 
UPDATE
F o o t b al l 
 News Watch Scores & FixturesTables Transfers More
25/07/2025, 19:28 Chelsea transfer news, rumours and gossip: Live updates and latest on deals, signings, loans and 
contracts | Football News | Sky Sports
https://www.skysports.com/football/live-blog/11668/13025497/chelsea-transfer-news-rumours-and-gossip-live-updates-a
nd-latest-on-deals-signings-loans-and-… 1/17

### Extract Information Using An LLM From The Document

- Extract relevant information from the document using an LLM before embedding.
- We'll be extracting the following:
  - A title
  - Potential questions and answers that can be answered using the document.
- Apply transformations to the documents.

In [11]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openrouter import OpenRouter

llm = OpenRouter(
    api_key=settings.OPENROUTER_API_KEY.get_secret_value(),
    max_tokens=2_048,
    context_window=4_096,
    model="meta-llama/llama-3.2-3b-instruct",
    temperature=0.1,
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [12]:
message = ChatMessage(role="user", content="Tell me a joke about Tyrion Lannister")
resp = llm.chat([message])
console.print(resp)

ChatResponse(
    message=ChatMessage(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        additional_kwargs={},
        blocks=[
            TextBlock(
                block_type='text',
                text='Why did Tyrion Lannister bring a ladder to the party?\n\nBecause he heard the drinks were on 
the house! (get it?)'
            )
        ]
    ),
    raw=ChatCompletion(
        id='gen-1753468505-NRaOZx6qJvzbVIzD6R16',
        choices=[
            Choice(
                finish_reason='stop',
                index=0,
                logprobs=None,
                message=ChatCompletionMessage(
                    content='Why did Tyrion Lannister bring a ladder to the party?\n\nBecause he heard the drinks 
were on the house! (get it?)',
                    refusal=None,
                    role='assistant',
                    annotations=None,
                    audio=None,
                    function_call=None,
                    tool_calls=None,
                    reasoning=None
                ),
                native_finish_reason='stop'
            )
        ],
        created=1753468505,
        model='meta-llama/llama-3.2-3b-instruct',
        object='chat.completion',
        service_tier=None,
        system_fingerprint=None,
        usage=CompletionUsage(
            completion_tokens=29,
            prompt_tokens=20,
            total_tokens=49,
            completion_tokens_details=None,
            prompt_tokens_details=None
        ),
        provider='Together'
    ),
    delta=None,
    logprobs=None,
    additional_kwargs={'prompt_tokens': 20, 'completion_tokens': 29, 'total_tokens': 49}
)

### [Data Ingestion](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/#ingestion-pipeline)

In [15]:
from llama_index.core.extractors import QuestionsAnsweredExtractor, TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(separator=" ", chunk_size=1024, chunk_overlap=128)
title_extractor = TitleExtractor(llm=llm, nodes=5)
qa_extractor = QuestionsAnsweredExtractor(llm=llm, questions=3)

ingestion_pipeline: IngestionPipeline = IngestionPipeline(
    # Order of execution
    transformations=[
        text_splitter,
        title_extractor,
        qa_extractor,
    ]
)

nodes = ingestion_pipeline.run(
    documents=docs,
    inplace=True,
    show_progress=True,
)

100%|██████████| 22/22 [00:30<00:00,  1.39s/it]


In [16]:
len(nodes)

22

In [17]:
console.print(nodes[0].get_content(metadata_mode=MetadataMode.EMBED))

[Excerpt from document]
file_path: /Users/neidu/Desktop/Projects/Personal/RAG-Tutorials/notebooks/../data/chelsea_transfer_news.pdf
document_title: Based on the provided title, I would suggest the following comprehensive title:

"Chelsea FC Transfer News, Fixtures, and Matchday Information for the 2025/26 English Premier League Season"

This title includes all the essential elements mentioned in the original title, such as:

* The team name (Chelsea FC)
* The specific type of content (Transfer News, Fixtures, and Matchday Information)
* The season (2025/26)
* The league (English Premier League)

This comprehensive title provides a clear and concise overview of the content, making it easier for users to find 
and understand the information they need.
questions_this_excerpt_can_answer: Here are three questions that this context can provide specific answers to, 
which are unlikely to be found elsewhere:

1. What is the latest update on Chelsea's transfer talks with RB Leipzig forward Xavi Simons, including the 
potential payment and other interested parties?

This question can be answered by reading the article excerpt, which provides an update on the transfer talks and 
quotes from Sky Germany's Leipzig reporter Philipp Hinze.

2. What is the title of the Sky Sports article that provides live updates and latest on Chelsea transfer news, 
rumours, and gossip?

This question can be answered by reading the article excerpt, which mentions the title of the article: "Chelsea 
transfer news, rumours and gossip: Live updates and latest on deals, signings, loans and contracts".

3. What is the URL of the Sky Sports article that provides live updates and latest on Chelsea transfer news, 
rumours, and gossip?

This question can be answered by reading the article excerpt, which mentions the URL of the article: 
"https://www.skysports.com/football/live-blog/11668/13025497/chelsea-transfer-news-rumours-and-gossip-live-updates-
and-latest-on-deals-signings-loans-and-…".

As for higher-level summaries of surrounding context, here are a few:

* The document appears to be a PDF file containing Chelsea FC transfer news, fixtures, and matchday information for
the 2025/26 English Premier League season.
* The document is likely to be a news article or a webpage from a sports website, given the presence of a live blog
and a URL.
* The document is dated July 25, 2025, which suggests that it is a recent publication, likely within the past few 
days or weeks.
Excerpt:
-----
Friday 25 July 2025 14:39, UK
Chelsea transfer news, rumours and
gossip: Live updates and latest on
deals, signings, loans and contracts
Latest Chelsea news 
Sort by: Latest Oldest
In full: Chelsea 2025/26 Premier League fixtures 
Transfer Centre LIVE! Deals, rumours, news on your phone
Download the Sky Sports app for Chelsea transfers, analysis and FREE highlights from
EVERY Premier League game   View post
24 Jul
16:22 Simons to Chelsea? The key questions answered...
Chelsea have held talks over signing RB Leipzig forward Xavi Simons - but how
much will they have to pay and who else want him? 
Sky Germany’s Leipzig reporter Philipp Hinze answers the key questions
around the deal. 
Keep scrolling! 
UPDATE
F o o t b al l 
 News Watch Scores & FixturesTables Transfers More
25/07/2025, 19:28 Chelsea transfer news, rumours and gossip: Live updates and latest on deals, signings, loans and 
contracts | Football News | Sky Sports
https://www.skysports.com/football/live-blog/11668/13025497/chelsea-transfer-news-rumours-and-gossip-live-updates-a
nd-latest-on-deals-signings-loans-and-… 1/17
-----